In [56]:
# Import dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os
from datetime import datetime
import requests
import pandas_datareader as pdr
from datetime import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%matplotlib inline
warnings.filterwarnings('ignore')

In [57]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
# Function to Validate Date
def validateShift(shift):
    shift = int(shift)
    # Try Check on Format
    try:
        if 0 > shift <= 365:
            pass
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Please enter a whole number.")

In [58]:
# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")

# Validate Correct Date Format
validate(date)

Pick a start date for your analysis (YYYY-MM-DD):  2019-08-07


In [59]:
# Pick a Stock
stock_selection = input("Pick a stock ticker for your analysis: ")

# Validate Correct Date Format
validateTick(stock_selection)

Pick a stock ticker for your analysis:  CME


In [60]:
# Pick Shift
shift_selection = input("Pick how many days to shift your close values for your analysis: ")

# Validate Correct Date Format
validateShift(shift_selection)

Pick how many days to shift your close values for your analysis:  5


In [61]:
# Function to Create Stocks
def stockCreation(date, stock_selection, days_shifted):
    
    # Read Stock Data
    globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
    # Create Shifted Close Column
    globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
    # Drop Rows Without Shift Values
    globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
    # Empty List to Hold Profit or Loss
    profit_loss = []
    
    # Iterate Through DataFrame
    for x in range(len(globals()[stock_selection])):
        
        # Handle Profitable Days
        if globals()[stock_selection]["Shifted Close"][x] > globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Profit" to Empty List
            profit_loss.append("Profit")
            
        # Handle Loss Days
        elif globals()[stock_selection]["Shifted Close"][x] < globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Loss" to Empty List
            profit_loss.append("Loss")
            
        # Handle Push Days
        elif globals()[stock_selection]["Shifted Close"][x] == globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Push" to Empty List
            profit_loss.append("Push")
    
    # Create Profit/Loss Column
    globals()[stock_selection]["Profit/Loss"] = profit_loss
    
    # Create Returns Column
    globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
    # Iterate to Create STD Columns
    for x in range(5, 20, 5):
        
        # Create STD Column Based on Iteration
        col_name = 'std_' + str(x)
        
        # Add STD Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

    # Iterate to Create MA Columns
    for x in range(10, 30, 5):
        
        # Create MA Column Based on Iteration
        col_name = 'ma_' + str(x)
        
        # Add MA Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

    # Iterate to Create % Change Columns
    for x in range(3, 12, 3):
        
        # Create % Change Column Based on Iteration
        col_name = 'pct_' + str(x)
        
        # Add % Change Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

    # Create VMA 4 Column
    col_name = 'vma_4'
    
    # Add VMA 4 Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

    # Create Intraday Column
    col_name = 'co'
    
    # Add Intraday Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
    # Drop All NaN's
    globals()[stock_selection].dropna(inplace=True)
    
    # Return DataFrame
    return globals()[stock_selection]

In [7]:
# ### Function to Create Stocks
# def stockCreation(date, stock_selection, days_shifted):
    
#     # Read Stock Data
#     globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
#     # Create Shifted Close Column
#     globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
#     # Drop Rows Without Shift Values
#     globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
#     # Empty List to Hold Profit or Loss
#     profit_loss = []
    
#     # Iterate Through DataFrame
#     for x in range(len(globals()[stock_selection])):
        
#         # Save Adjusted Close Value
#         adj_close = globals()[stock_selection]["Adj Close"][x]
        
#         # Save Shifted Close Value
#         shifted_close = globals()[stock_selection]["Shifted Close"][x]
        
#         # Handle Returns 3x as Large as Open
#         if shifted_close > (3 * adj_close):
            
#             # Append Number 8
#             profit_loss.append(8)
            
#         # Handle Returns within 2x and 3x as Large as Open
#         elif (3 * adj_close) <= shifted_close > (2 * adj_close):
            
#             # Append Number 7
#             profit_loss.append(7)
            
#         # Handle Returns within 1.75x and 2x as Large as Open
#         elif (2 * adj_close) <= shifted_close > (1.75 * adj_close):
            
#             # Append Number 6 
#             profit_loss.append(6)
        
#         # Handle Returns within 1.5x and 1.75x as Large as Open
#         elif (1.75 * adj_close) <= shifted_close > (1.5 * adj_close):
            
#             # Append Number 5
#             profit_loss.append(5)
            
#         # Handle Returns within 1,25x and 1.5x as Large as Open
#         elif (1.5 * adj_close) <= shifted_close > (1.25 * adj_close):
            
#             # Append Number 4
#             profit_loss.append(4)
            
#         # Handle Returns within 1.1x and 1.25x as Large as Open
#         elif (1.25 * adj_close) <= shifted_close > (1.1 * adj_close):
            
#             # Append Number 3
#             profit_loss.append(3)
            
#         # Handle Returns within 1.05x and 1.1x as Large as Open
#         elif (1.1 * adj_close) <= shifted_close > (1.05 * adj_close):
            
#             # Append Number 2
#             profit_loss.append(2)
            
#         # Handle Returns within 1x and 1.05x as Large as Open
#         elif (1.05 * adj_close) <= shifted_close > (1 * adj_close):
            
#             # Append Number 1
#             profit_loss.append(1)
            
#         # Handle Returns within .95x and 1x as Small as Open
#         elif (1 * adj_close) <= shifted_close > (.95 * adj_close):
            
#             # Append Number 0
#             profit_loss.append(0)
        
#         # Handle Returns within .9x and .95x as Small as Open
#         elif (.95 * adj_close) <= shifted_close > (.9 * adj_close):
            
#             # Append Number -1
#             profit_loss.append(-1)
            
#         # Handle Returns within .75x and .9x as Small as Open
#         elif (.9 * adj_close) <= shifted_close > (.75 * adj_close):
            
#             # Append Number -2
#             profit_loss.append(-2)
            
#         # Handle Returns within .5x and .75x as Small as Open
#         elif (.75 * adj_close) <= shifted_close > (.5 * adj_close):
            
#             # Append Number -3
#             profit_loss.append(-3)
            
#         # Handle Returns within .25x and .5x as Small as Open
#         elif (.5 * adj_close) <= shifted_close > (.25 * adj_close):
            
#             # Append Number -4
#             profit_loss.append(-4)
            
#         # Handle Returns within .1x and .25x as Small as Open
#         elif (.25 * adj_close) <= shifted_close > (.1 * adj_close):
            
#             # Append Number -5
#             profit_loss.append(-5)
            
#         # Handle Returns within .05x and .1x as Small as Open
#         elif (.1 * adj_close) <= shifted_close > (.05 * adj_close):
            
#             # Append Number -6
#             profit_loss.append(-6)
        
#         # Handle Returns Smaller than .05x as Small as Open
#         elif shifted_close < (.05 * adj_close):
            
#             # Append Number -7
#             profit_loss.append(-7)
    
#     # Create Profit/Loss Column
#     globals()[stock_selection]["Profit/Loss"] = profit_loss
    
#     # Create Returns Column
#     globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
#     # Iterate to Create STD Columns
#     for x in range(5, 20, 5):
        
#         # Create STD Column Based on Iteration
#         col_name = 'std_' + str(x)
        
#         # Add STD Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

#     # Iterate to Create MA Columns
#     for x in range(10, 30, 5):
        
#         # Create MA Column Based on Iteration
#         col_name = 'ma_' + str(x)
        
#         # Add MA Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

#     # Iterate to Create % Change Columns
#     for x in range(3, 12, 3):
        
#         # Create % Change Column Based on Iteration
#         col_name = 'pct_' + str(x)
        
#         # Add % Change Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

#     # Create VMA 4 Column
#     col_name = 'vma_4'
    
#     # Add VMA 4 Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

#     # Create Intraday Column
#     col_name = 'co'
    
#     # Add Intraday Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
#     # Drop All NaN's
#     globals()[stock_selection].dropna(inplace=True)
    
#     # Return DataFrame
#     return globals()[stock_selection]

In [62]:
# Create Stock
stockdf = stockCreation(date, stock_selection, shift_selection)
stockdf.head()

,High,Low,Open,Close,Volume,Adj Close,Shifted Close,Profit/Loss,Returns,std_5,...,std_15,ma_10,ma_15,ma_20,ma_25,pct_3,pct_6,pct_9,vma_4,co
Date,,,,,,,,,,,,,,,,,,,,,
2019-09-11,206.699997,201.940002,206.000000,202.580002,2564200.0,193.065155,206.016846,Profit,-0.013970,8.448409,...,5.034868,204.925609,204.296075,203.808915,203.367687,-0.091272,-0.067030,-0.057773,1892750.0,-12.934845
2019-09-12,205.770004,201.779999,201.979996,205.380005,1764800.0,195.733673,201.995087,Profit,0.013727,7.971804,...,5.494165,204.005501,203.826411,203.701247,203.260656,-0.047134,-0.068363,-0.043627,2060475.0,-6.246323
2019-09-13,206.539993,202.539993,206.130005,205.800003,1122600.0,196.133926,201.232635,Profit,0.002043,4.757320,...,5.839445,203.132066,203.399258,203.406052,203.085670,0.001993,-0.066860,-0.049000,2005325.0,-9.996078
2019-09-16,209.190002,205.130005,205.570007,207.000000,1244700.0,197.277573,202.309570,Profit,0.005814,1.547214,...,6.014308,202.221027,203.176259,203.126724,202.780903,0.021698,-0.069575,-0.045332,1674075.0,-8.292435
2019-09-17,214.860001,205.630005,207.020004,214.729996,2497000.0,204.644485,202.909988,Loss,0.036663,4.349247,...,6.023999,202.002034,203.227829,203.067568,202.902646,0.045219,-0.001915,-0.023144,1657275.0,-2.375519


In [63]:
# # change profit/loss to 0's/1's
# ## convert from string to integer
profit_loss = []
for row in stockdf.iloc[:,7]:
    if row == 'Loss':
         profit_loss.append(0)
    else:
         profit_loss.append(1)
            
stockdf ["Profit/Loss"] = profit_loss

In [79]:
stockdf.info

<bound method DataFrame.info of                   High         Low        Open       Close     Volume  \
Date                                                                    
2019-09-11  206.699997  201.940002  206.000000  202.580002  2564200.0   
2019-09-12  205.770004  201.779999  201.979996  205.380005  1764800.0   
2019-09-13  206.539993  202.539993  206.130005  205.800003  1122600.0   
2019-09-16  209.190002  205.130005  205.570007  207.000000  1244700.0   
2019-09-17  214.860001  205.630005  207.020004  214.729996  2497000.0   
...                ...         ...         ...         ...        ...   
2021-02-04  194.839996  192.160004  192.940002  193.529999  1394300.0   
2021-02-05  194.449997  188.429993  194.089996  191.389999  1741400.0   
2021-02-08  193.669998  190.240005  192.940002  191.740005  2071100.0   
2021-02-09  195.539993  191.229996  193.000000  191.970001  2026300.0   
2021-02-10  192.190002  182.660004  191.119995  185.199997  2688300.0   

             Adj C

In [80]:
# Create function for column numbering including features (X) and target (y)
def window_data(df, window, feature_column, target_column):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_column]
        target = df.iloc[(i + window), target_column]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [81]:
# Predict Closing Prices
window_size = 100

feature_column = 7
target_column = 1
X, y = window_data(stockdf, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0
  0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0
  0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0
  0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0
  1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0
  1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1
  1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1
  1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1
  1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1
  1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1
  1 1 1

In [66]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1 ]
X_test = X[split:]
y_train = y[: split - 1 ]
y_test = y[split:]

In [67]:
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
X_train[:5]

array([[1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
        0., 1.

In [68]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]} \n")

X_train sample values:
[[[1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  

In [69]:
# Build the LSTM model. 

model = Sequential()

number_units = 100
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1,activation='sigmoid'))

In [70]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy")

In [71]:
# Summarize the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100, 100)          40800     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100, 100)          80400     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 100)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [72]:
# Train the model
model.fit(X_train, y_train, epochs=50, shuffle=False, batch_size=5, verbose=1)

Epoch 1/50
36/36 [==============================] - 6s 64ms/step - loss: 0.7253
Epoch 2/50
36/36 [==============================] - 2s 64ms/step - loss: 0.7283
Epoch 3/50
36/36 [==============================] - 2s 62ms/step - loss: 0.6958
Epoch 4/50
36/36 [==============================] - 2s 61ms/step - loss: 0.6919
Epoch 5/50
36/36 [==============================] - 2s 64ms/step - loss: 0.6927
Epoch 6/50
36/36 [==============================] - 2s 63ms/step - loss: 0.6935
Epoch 7/50
36/36 [==============================] - 2s 66ms/step - loss: 0.6924
Epoch 8/50
36/36 [==============================] - 2s 63ms/step - loss: 0.6925
Epoch 9/50
36/36 [==============================] - 2s 62ms/step - loss: 0.6930
Epoch 10/50
36/36 [==============================] - 2s 63ms/step - loss: 0.6935
Epoch 11/50
36/36 [==============================] - 2s 62ms/step - loss: 0.6941
Epoch 12/50
36/36 [==============================] - 2s 62ms/step - loss: 0.6945
Epoch 13/50
36/36 [==================

In [73]:
# Evaluate the model
model.evaluate(X_test, y_test)

3/3 [==============================] - 1s 30ms/step - loss: 0.6829


0.6829241514205933

In [74]:
# Predicting
predicted = model.predict(X_test)

In [75]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [78]:
# Create a DataFrame of Real and Predicted values
stockdf = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = df.index[-len(real_prices): ]) 
stockdf.head()

ValueError: Shape of passed values is (78, 2), indices imply (0, 2)

In [49]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(title="Real vs Predicted Stock Price", xlabel="Days", ylabel="Price")

NameError: name 'stocks' is not defined

In [ ]:
# # change profit/loss to 0's/1's
# ## convert from string to integer
# profit_loss = []
# for row in stockdf.iloc[:,7]:
#     if row == 'Loss':
#         profit_loss.append(0)
#     else:
#         profit_loss.append(1)

In [ ]:
# stockdf['Profit/Loss'] = profit_loss

In [ ]:
# # Assign Y Values
# y = stockCreation(date, stock_selection, shift_selection)["Profit/Loss"]

In [ ]:
# # Assign X Values
# X = stockCreation(date, stock_selection, shift_selection).drop(["Shifted Close", "Profit/Loss"], axis=1)

In [ ]:
# # Train/Test Split 
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# # Assign Y Values
# y = stockCreation(date, stock_selection, shift_selection)["Profit/Loss"]

In [ ]:
# # Assign X Values
# X = stockCreation(date, stock_selection, shift_selection).drop(["Shifted Close", "Profit/Loss"], axis=1)

In [ ]:
# # Train/Test Split 
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)